In [ ]:
from astropy.io import fits as fits
import numpy as np

In [ ]:
def averaging (path, x, y, type, total_frames):
    
    comb = np.zeros((x,y))
    
    for j in range(total_frames):
        comb += fits.getdata(path + type + '_' + str(j+1) + '.fit')
        
    avg = comb/total_frames
    
    return avg

In [ ]:
frames_in_set = 10
object_sets = 12
star_sets = 6
dark_sets = 7

object_frames = object_sets * frames_in_set
star_frames = star_sets * frames_in_set
dark_frames = dark_sets * frames_in_set

x = 1039
y = 1391

path_raw = 'C:/Users/Calum/Documents/My Documents/354_Eleonora_Flyby2/Actual_Data/Initial_Frames/'
path_clean = 'C:/Users/Calum/Documents/My Documents/354_Eleonora_Flyby2/Actual_Data/Cleaned_Frames/'
path_comb = 'C:/Users/Calum/Documents/My Documents/354_Eleonora_Flyby2/Actual_Data/Combined_Frames/'

In [ ]:
# Takes an average of all the dark frames
avg_dark = averaging(path_raw, x, y, 'dark', dark_frames)

In [ ]:
# Removes the average dark frame from all the asteroid and star frames

for j in range(object_frames):
    with fits.open(path_raw + 'object' + '_' + str(j+1) + '.fit') as hdul:
        data = (hdul[0].data)/1.0
        data -= avg_dark
        hdul.writeto(path_clean + 'object' + '_' + str(j+1) + '.fit')
    if (j+1) % 10 == 0:
        print('Object ' + str(j+1))
    
for j in range(star_frames):
    with fits.open(path_raw + 'star' + '_' + str(j+1) + '.fit') as hdul:
        data = (hdul[0].data)/1.0
        data -= avg_dark
        hdul.writeto(path_clean + 'star' + '_' + str(j+1) + '.fit')
    if (j+1) % 10 == 0:
        print('Star ' + str(j+1))

In [ ]:
# Group the asteroid and star frames into combinations of 10 frames each

for j in range(object_sets):
    comb = np.zeros((x,y))
    for k in range(frames_in_set):
        with fits.open(path_clean + 'object' + '_' + str(j*frames_in_set+k+1) + '.fit') as hdul:
            data = hdul[0].data
            comb += data
    image = comb / frames_in_set
    hdu = fits.PrimaryHDU(image)
    hdu.writeto(path_comb + 'object' + '_' + str(j+1) + '.fit')
    
for j in range(star_sets):
    comb = np.zeros((x,y))
    for k in range(frames_in_set):
        with fits.open(path_clean + 'star' + '_' + str(j*frames_in_set+k+1) + '.fit') as hdul:
            data = hdul[0].data
            comb += data
    image = comb / frames_in_set
    hdu = fits.PrimaryHDU(image)
    hdu.writeto(path_comb + 'star' + '_' + str(j+1) + '.fit')